In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from dataset import extract_users_movies_ratings_lists, TripletDataset, extract_matrix_users_movies_ratings
import torch 

#Useful constants
number_of_users, number_of_movies = (10000, 1000)
RANDOM_STATE = 58
DATA_DIR = '../data'


data_pd = pd.read_csv(DATA_DIR+'/data_train.csv')
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)


#users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
#users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
matrix_train, _ = extract_matrix_users_movies_ratings(train_pd)
matrix_val, _ = extract_matrix_users_movies_ratings(val_pd)

matrix_train = torch.tensor(matrix_train)
matrix_val = torch.tensor(matrix_val)

In [55]:
K=6


In [56]:
C = 1e-7
def step(u, v, y):
    u_new = torch.empty_like(u)
    v_new = torch.empty_like(v)

    uv = torch.matmul(u,v.transpose(0,1)) + C
    a = torch.matmul(v.transpose(0,1), (y/uv).transpose(0,1)) #K,num_users
    b = torch.sum(v, dim=0) + C #K 
    
    c = a.transpose(0,1)/b
    u_new = u * c
            
        
    u = u_new

    uv = torch.matmul(u,v.transpose(0,1))+ C
    a = torch.matmul(u.transpose(0,1), (y/uv)) #K, num_movies
    b = torch.sum(u, dim=0) + C #K

   
    c = a.transpose(0,1) / b
   
    v_new = v * c
     
            
    return u_new, v_new


In [57]:
def rmse(y, yhat):
    mask = y.to(torch.bool)
    return torch.sqrt(torch.mean((y-yhat)[mask]**2))

In [58]:
matrix_train.type()

'torch.FloatTensor'

In [59]:
u = torch.rand((number_of_users,K)) + 1e-5
v = torch.rand((number_of_movies, K)) + 1e-5


old_score = torch.inf
n = 0
while True:
    u, v = step(u, v, matrix_train)
    pred = torch.matmul(u,v.transpose(0,1))
    val_score = rmse(matrix_val, pred)
    train_score = rmse(matrix_train, pred)

   
    print(f'Train rmse: {train_score}, val rmse: {val_score}')
    continue
    if n % 1000 == 0:
        if val_score < old_score:
            np.save('./best_u', u)
            np.save('./best_v', v)
            old_score = val_score
        np.save('./u', u)
        np.save('./v', v)
    n+=1


Train rmse: 3.1737983226776123, val rmse: 3.183397054672241
Train rmse: 3.1719577312469482, val rmse: 3.1834187507629395
Train rmse: 3.1704487800598145, val rmse: 3.183515787124634
Train rmse: 3.1691510677337646, val rmse: 3.1836535930633545
Train rmse: 3.167990207672119, val rmse: 3.183810234069824
Train rmse: 3.1669139862060547, val rmse: 3.1839709281921387
Train rmse: 3.165884494781494, val rmse: 3.1841208934783936
Train rmse: 3.164872169494629, val rmse: 3.184246063232422
Train rmse: 3.163848638534546, val rmse: 3.1843297481536865
Train rmse: 3.1627864837646484, val rmse: 3.1843514442443848
Train rmse: 3.161656379699707, val rmse: 3.1842854022979736
Train rmse: 3.1604247093200684, val rmse: 3.184100389480591
Train rmse: 3.159053087234497, val rmse: 3.1837570667266846
Train rmse: 3.1574978828430176, val rmse: 3.183210849761963
Train rmse: 3.1557118892669678, val rmse: 3.1824111938476562
Train rmse: 3.153648614883423, val rmse: 3.1813063621520996
Train rmse: 3.151268720626831, val rm

KeyboardInterrupt: 